# 1. 项目地址  
https://python.langchain.com/docs/tutorials/llm_chain/

通过这个项目，你将获得的能力：  
1.使用大语言模型  
2.使用提示词模版  
3.使用LangSmith来调试和追踪你的应用问题  

# 2. 代码环境设置
Jupyter Notebook or Jupyter Lab
自行找文档下载安装哈

## 安装langchain

pip install langchain

In [ ]:
import langchain

# 3. 设置Langsmith
1. LangSmith是做什么的  
    - 跟踪调试：记录模型运行过程，定位问题。
    - 测试评估：批量测试模型表现，支持自定义标准。
    - 实时监控：跟踪生产环境性能和用户交互。
    - Prompt 管理：优化和管理提示词，支持版本控制。  
    
2. Address：
    https://smith.langchain.com/onboarding?organizationId=d33de966-2acd-4e99-884a-4a51a8600109&step=1
3. 其他:
    - 需要注册
    - 需要生成API key用来交互，服务在云端
    - 如没有监控需求，也可以选择不启用这个监控服务，对后续无影响

In [2]:
# 代码

import getpass
import os  

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()  

 ········


**代码解释**
1. getpass：getpass 是一个用于安全输入密码或敏感信息的工具。它的主要功能是让用户在终端输入内容时不回显（即输入时不会显示字符，通常用于密码输入）。
2. os：os 模块提供了与操作系统交互的功能，包括文件操作、路径处理和环境变量管理等。
3. os.environ["LANGSMITH_TRACING"] = "true" 用来激活LangSmith服务，在后续的Langchain模块中会检测这个环境变量是否启用，如果启用会和LangSmith服务进行交互进而开启监控。
4. os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
    1. 用来拿到LangSmith的API Key
    2. getpass.getpass() 会提示用户在终端输入内容（默认提示是 "Password: "），输入的内容不会显示在屏幕上
    3. Appkey需要注册账号，然后在网站内生成即可，也可以选择不启用这个监控服务，跳过即可。

# 4. 使用大语言模型

**安装langchain的集成**  
pip install -qU "langchain-deepseek"

- 安装Deepseek的Langchain扩展，用于和Deepseek接口进行交互。
- Langchain也可以使用其他的模型，具体请直接查询官方文档代码，集成会有不一样，代码会有所变化。

In [3]:
# 前置需要进入deepseek官网注册并生成API_KEY，记得保存好，否则丢失后需要重新生成。
# 这一行检查程序是否已经有 Deepseek API 密钥。如果没有，就需要用户手动输入。
if not os.getenv("DEEPSEEK_API_KEY"):
    os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("Enter your DeepSeek API key: ")

Enter your DeepSeek API key:  ········


**代码解释**
- os.getenv(key) 是 os 模块的一个函数，用于读取环境变量的值。
- 如果环境变量 DEEPSEEK_API_KEY 已设置（比如通过终端的 export DEEPSEEK_API_KEY="your-key"），它返回对应的值（字符串）。
- 如果未设置或值为空，则返回 None。


In [4]:
# 配置接口参数，创建语言模型的实例
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(
    model="deepseek-chat",  # 指定使用的模型
    temperature=0,  # 控制生成文本的随机性
    max_tokens=None,  # 生成的最大 token 数限制
    timeout=None,  # 请求超时时间
    max_retries=2, # 请求失败时的最大重试次数
    # other params...
)

- 让我们使用模型试试。ChatModels是LangChain中可运行的组件，这意味着他们会有一个标准接口来进行交户。唤起模型很简单，我们给.invoke方法传送一列消息即可达到。

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

llm.invoke(messages)

AIMessage(content='Ciao!  \n\n(If you\'re looking for a more formal greeting, you could also say "Salve!") 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 12, 'total_tokens': 38, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 12}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3d5141a69a_prod0225', 'finish_reason': 'stop', 'logprobs': None}, id='run-4b103a31-b533-4d6a-8437-5d55c5b447fd-0', usage_metadata={'input_tokens': 12, 'output_tokens': 26, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

- 注意，大语言模型会接受消息对象作为输入然后生成消息对象作为输出。除了文本内容外，消息对象也可以表达交流角色并包含重要数据，例如工具调用和字数统计。

- LangChain也支持聊天模型的输入通过字符串模式或者OpenAI 格式。以下是一下例子：

In [30]:
llm.invoke("Hello")
llm.invoke([{"role":"user","content":"Hello"}])
llm.invoke([HumanMessage("Hello")])

AIMessage(content='Hello! How can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 4, 'total_tokens': 15, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 4}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3a5770e1b4_prod0225', 'finish_reason': 'stop', 'logprobs': None}, id='run-cbd59df7-eccf-4005-a67a-2878f9fb85e7-0', usage_metadata={'input_tokens': 4, 'output_tokens': 11, 'total_tokens': 15, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [31]:
llm.invoke("Hello")

AIMessage(content='Hello! How can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 4, 'total_tokens': 15, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 4}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3a5770e1b4_prod0225', 'finish_reason': 'stop', 'logprobs': None}, id='run-dd36534c-0381-4ef0-a5bf-50ad38781363-0', usage_metadata={'input_tokens': 4, 'output_tokens': 11, 'total_tokens': 15, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [32]:
llm.invoke([{"role":"user","content":"Hello"}])

AIMessage(content='Hello! How can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 4, 'total_tokens': 15, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 4}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3a5770e1b4_prod0225', 'finish_reason': 'stop', 'logprobs': None}, id='run-fe1f08eb-146a-4b76-806e-d71137c72bea-0', usage_metadata={'input_tokens': 4, 'output_tokens': 11, 'total_tokens': 15, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [33]:
llm.invoke([HumanMessage("Hello")])

AIMessage(content='Hello! How can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 4, 'total_tokens': 15, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 4}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3a5770e1b4_prod0225', 'finish_reason': 'stop', 'logprobs': None}, id='run-65d3613b-51be-4613-b4d6-ae2374ef3277-0', usage_metadata={'input_tokens': 4, 'output_tokens': 11, 'total_tokens': 15, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

# 5. 流式处理stream

- 因为聊天模型是可运行的组件-Runnables，他们有一个标准的接口，这个接口包含异步和流式调用模型。这就使得我们可以使用流式的方式处理单个字符。

In [38]:
llm.stream(messages)

<generator object BaseChatModel.stream at 0x7f7b218af660>

In [37]:
# 流式处理
for token in llm.stream(messages):
    print(token.content, end="|")

|C|iao|!||

**代码解释**
- 通过 llm.stream(messages) 调用语言模型的流式输出方法，并用 for 循环迭代处理返回的每个 token。
- stream 是语言模型提供的一个方法，用于以流式（streaming）方式生成输出。
- 与一次性返回完整结果的 invoke 不同，stream 返回一个迭代器（iterator）或生成器（generator），逐步生成并返回模型输出的每个 token。

# 6. 提示词模版

- 我们把一列信息直接给到大语言模型。这一列信息一般由用户输入和应用逻辑共同组成。这个应用逻辑通常拿到原始的用户输入并把这个输入转化成一组组织好的消息准备传送给大语言模型。一般的用户信息形式转换涉及结合用户输入添加系统信息或者构建一个模版。
- LangChain就是这样来设计提示词模版概念来帮助实现这一转化过程。首先拿到用户的原始输入信息，然后转化为一个可以直接传输给大模型的提示词模版。
- 我们在这里构建一个提示词模版样例，这将会使用两个用户变量：

In [39]:
# 定义一个提示模板，用于将用户输入的英文文本翻译成指定语言并传递给大模型。
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system",system_template),("user","{text}")]
)


**代码解释**
- ChatPromptTemplate 是 LangChain 提供的一个工具，用于构建结构化的对话提示模板。
- 它特别适合与支持聊天格式的语言模型（如 ChatDeepSeek、ChatOpenAI）配合使用，允许定义多角色消息（如系统消息、用户消息）。
- prompt_template 是一个可重用的模板对象，允许动态填入 {language} 和 {text} 的值，生成完整的提示。

- 注意，ChatPromptTemplate支持多种消息类型在单一模版中。我们将“language”参数嵌入到“system”消息中，将“text”嵌入到了“user”消息中。
- 这个提示词模版的输入是一个字典，我们下面使用这个模版看看能生成什么结果：

In [40]:
# 这一步只是替换language和text占位符的内容为具体的值，
# 后续可以使用prompt作为最终提示词来提交给大模型使用
# prompt_template在这里就成立一个包含参数的灵活的提示词模版了
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

**代码解释**
- invoke 是 ChatPromptTemplate 的方法，用于将占位符替换为具体值，生成一个具体的提示对象。
- 输入是一个字典，键（language 和 text）对应模板中的占位符，值（"Italian" 和 "hi!"）是替换的内容。

In [15]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

**代码解释**
- 调用 prompt 对象的 to_messages() 方法，返回一个消息列表。

In [41]:
#与大模型交互，将提示词模版传输给大模型，并得到答案
response = llm.invoke(prompt)
print(response.content)

Ciao!


# 7. 总结
- 到这里就结束了，你已经学会了如何创建你的第一个简单的大语言模型应用。又学习了如何与大语言模型进行交互，如何创造一个提示词模版，并且如何使用LangSmith去追踪你应用的运行情况。
- 这对于你成一个专业的AI工程师只是隔靴搔痒，我们还有其他很多资源可供学习！